# Simple character level tokenizer

In [466]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("\nVocabulary size is ", vocab_size, "chars/tokens: ", ''.join(chars))

ctoi, itoc = dict((char, index) for index, char in enumerate(chars)), dict((index, char) for index, char in enumerate(chars))
print("\nVocabulary: ",  ctoi)

encode = lambda sentence: [ctoi[char] for char in sentence] # encode sentence to index
decode = lambda indexes: ''.join([itoc[index] for index in indexes]) # decode index to sentence
print("example: ",encode("some text for training"), decode(encode("some text for training")))


Vocabulary size is  65 chars/tokens:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

Vocabulary:  {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
example:  [57, 53, 51, 43, 1, 58, 43, 62, 58, 1, 44, 53, 56, 1, 58, 56, 39, 47, 52, 47, 52, 45] some text for training


In [467]:
# # WORD LEVEL
# with open('more.txt', 'r') as f:
#     text = f.read()
# words = sorted(list(set(text.split(' '))))
# vocab_size = len(words)

# print("Vocabulary size: ", len(words), "words/tokens")

# wtoi, itow = dict((word, index) for index, word in enumerate(words)), dict((index, word) for index, word in enumerate(words))
# print("\nVocabulary: ",  wtoi)

# encode = lambda sentence: [wtoi[word] for word in sentence.split(' ')] # encode sentence to index
# decode = lambda indexes: ' '.join([itow[index] for index in indexes]) # decode index to sentence
# # print("example: ",encode("some text for training"), decode(encode("some text for training")))

In [468]:
import torch
data = torch.LongTensor(encode(text))
print("\nData shape:", data.shape, "\nData type:", data.dtype)
print("Data example:\n", data[:])


Data shape: torch.Size([1115393]) 
Data type: torch.int64
Data example:
 tensor([18, 47, 56,  ..., 52, 45,  8])


# Separate data to train and validation 90/10
### generate a small batch of data of inputs x and targets y

In [469]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [470]:
# block size
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input is {context} and target is {target}")

input is tensor([18]) and target is 47
input is tensor([18, 47]) and target is 56
input is tensor([18, 47, 56]) and target is 57
input is tensor([18, 47, 56, 57]) and target is 58
input is tensor([18, 47, 56, 57, 58]) and target is 1
input is tensor([18, 47, 56, 57, 58,  1]) and target is 15
input is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58


In [471]:
# Batch size dimension
torch.manual_seed(674815) # for reproducibility (same random numbers every time)
batch_size = 4 # number of independent sequences to train on in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"input is {context.tolist()} and target is {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [57, 10,  0, 21,  1, 61, 47, 50],
        [44, 43,  1, 40, 63,  1, 58, 46],
        [ 5, 42, 10,  1, 41, 39, 50, 50]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 52],
        [10,  0, 21,  1, 61, 47, 50, 50],
        [43,  1, 40, 63,  1, 58, 46, 43],
        [42, 10,  1, 41, 39, 50, 50, 47]])
----
input is [24] and target is 43
input is [24, 43] and target is 58
input is [24, 43, 58] and target is 5
input is [24, 43, 58, 5] and target is 57
input is [24, 43, 58, 5, 57] and target is 1
input is [24, 43, 58, 5, 57, 1] and target is 46
input is [24, 43, 58, 5, 57, 1, 46] and target is 43
input is [24, 43, 58, 5, 57, 1, 46, 43] and target is 52
input is [57] and target is 10
input is [57, 10] and target is 0
input is [57, 10, 0] and target is 21
input is [57, 10, 0, 21] and target is 1
input is [57, 10, 0, 21, 1] and target is 61
input is [57, 10, 0, 21, 1, 61] and target is 47
input is [57, 

In [472]:
print(xb) # input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [57, 10,  0, 21,  1, 61, 47, 50],
        [44, 43,  1, 40, 63,  1, 58, 46],
        [ 5, 42, 10,  1, 41, 39, 50, 50]])


# Bigram language Model (simplest nn)

In [473]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # for reproducibility (same random numbers every time)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        # each token directly reads off the logits for the next token from a lookup table
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # compute the loss
            # loss = F.cross_entropy(logits.view(-1, vocab_size), target.view(-1))
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # 2D tensor for pytorch cross-entropy
            targets = targets.view(B*T) # 1D tensor
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # appendsampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
        
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, '\nloss is',loss)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) 
loss is tensor(4.8136, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


# create torch optimizer

In [474]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #0.001)

batch_size = 32
for steps in range(1_000):
    # semple a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

3.666220188140869


In [475]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Wh;;Sq.f ustNzknc
kwgOj$dhPWr,SV?hsusiKpgXXUh;Apmem d?hESXI.i;TrJgkiF-oKbXCAA -botrngFCHAUQkn$

pn$w-gHoi?wtd!
LLULIfSK'bAw :M.ZtOptXEQcL?hfaofqbPd?OnonQQJMap$aypupIBYGUsZaI'ottllo..k$W$Akp?yl?ajKlzY!lx&QQLW? t,bXFkyhl-dmVsHUEcORl,jSClgjuk:3Iv
?OqlrV;!Plxfzgy;;
'mRWvkQ&xk!$
h
SiruDJgKuDny,S$ERf.?GSV-ivvKcOvi-nQGX&q-YQbm dEM?px;Akr-IEmZ&-wIWId
RFgXTpDUgM:CK$I!uo'IBT -
j?wfy fFr.&fiqtRS.ZttxGh KG'dg


# Math trick in self attention

In [476]:
# toy example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch size, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [477]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for _ in range(10000): # banchmarking
    for b in range(B):
        for t in range(T):
            xprev = x[b, :t+1] # (t, C)
            xbow[b, t] = torch.mean(xprev, 0)
# xbow

In [478]:
torch.tril(torch.ones((10, 10)))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [479]:
# version 2. Same but efficient with trailing dimensions
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(1, keepdims=True)
for _ in range(10000): # banchmarking
    xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
# xbow2

In [480]:
# check that they are the same
torch.allclose(xbow, xbow2)

True

In [481]:
torch.manual_seed(42)
# a = torch.ones(3,3)
# make average pooling matrix
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [482]:
# version 3: use Softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

True

In [483]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch size, time, channels
x = torch.randn(B, T, C)

# single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, H=16)
q = query(x) # (B, T, H=16)

# compute attention weights (communication matrix)
wei = q @ k.transpose(-2, -1) # (B ,T, 16) @ (B, 16, T) -> (B, T, T)

# past/present communication between tokens
tril = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape 

torch.Size([4, 8, 16])

In [484]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [485]:
wei.var(dim=0)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.6402e-02, 5.6402e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.6099e-03, 9.4913e-02, 9.8885e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.3368e-02, 9.0478e-02, 1.9537e-02, 3.5656e-03, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.5143e-01, 4.1616e-02, 1.0740e-04, 1.2179e-01, 1.2508e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.7917e-03, 8.3901e-02, 1.8765e-03, 8.1908e-02, 7.5809e-02, 1.7662e-02,
         0.0000e+00, 0.0000e+00],
        [5.0183e-03, 2.8238e-02, 1.3197e-03, 1.7096e-02, 2.3426e-03, 3.1245e-02,
         2.1520e-03, 0.0000e+00],
        [2.5585e-02, 9.8163e-03, 3.8367e-04, 6.9569e-03, 1.1476e-02, 8.6107e-04,
         1.0128e-02, 9.0956e-03]], grad_fn=<VarBackward0>)

#  Attention divided by sqrt(d_k) for numerical stability

In [486]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
wei.shape

torch.Size([4, 8, 8])

In [487]:
wei.var()

tensor(1.0918)